# Classification Problem
Next I will develop a Logistic Regression model to predict different classes. More specifically Logistic Regression is used to estimate the probability that an instance/element/observation belongs to a certain class. The use of one the most popular collections of information for the purpose of classification is the Titanic dataset and the model I will develop is the one I have initially chosen to submit to Kaggle's 'Titanic' competition.

## Import the Python Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve
%matplotlib inline

## Import the Data